In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
movie = pd.read_csv("movie.csv")
# "rating.csv" dosyasını okuyarak bir veri çerçevesi oluşturur.
# Ardından "timestamp" sütununu veriden çıkarır.
rating = pd.read_csv("rating.csv").drop("timestamp",axis=1)

In [ ]:
print(movie.dtypes)
print(rating.dtypes)

In [ ]:
movie["movieId"] = movie["movieId"].astype(object)
rating["movieId"] = rating["movieId"].astype(object)
rating["userId"] = rating["userId"].astype(object)


In [ ]:
print(movie.isnull().sum())
print(rating.isnull().sum())

In [ ]:
C = rating["rating"].mean()
print(C)

In [ ]:
# Veri çerçevesindeki "movieId" sütununa göre gruplama yapılır ve her bir film için oy sayısını hesaplar.
rating_count = rating.groupby(["movieId"]).count()

# Yeni bir "movieId" sütunu oluşturarak gruplama sonuçlarını bu sütunda saklar.
rating_count["movieId"] = rating_count.index

# İndeks adını "None" olarak değiştirir.
rating_count.index.name = None

# İndeksi sıfırlayarak, yeni "movieId" sütununu eski indeks olarak kullanır.
rating_count.reset_index(inplace=True)

# Gereksiz "index" sütununu kaldırır.
del rating_count["index"]

rating_count



In [ ]:
means = rating.groupby(["movieId"]).mean()
means["movieId"] = means.index
means.index.name = None
means.reset_index(inplace=True)
del means["index"]
means = means.rename(columns={"rating":"mean"})

means

In [ ]:
df = pd.merge(movie, rating_count)
df = pd.merge(df, means)

print(df.head)

## persentilde ( yüzdelik dilim )

In [ ]:
# Veri çerçevesindeki "userId" sütunundaki değerlerin %90'ını belirleyerek eşik değeri oluşturur.
m = df["userId"].quantile(0.90)

# Orijinal veri çerçevesini kopyalar ve sadece "userId" sütunundaki değeri eşik değerden büyük veya eşit olanları içerecek şekilde filtreler.
q_movies = df.copy().loc[df["userId"] >= m]

q_movies.shape


In [ ]:
# Bir film için ağırlıklı bir puan hesaplamak üzere kullanılacak fonksiyonu tanımlar.
# Bu fonksiyon, kullanıcı sayısı (v), ortalama puan (R), eşik değeri (m) ve normalleştirme sabiti (C) parametrelerini alır.
def weight_rating(x, m=m, C=C):
    v = x["userId"]  # Kullanıcı sayısını alır.
    R = x["mean"]    # Ortalama puanı alır.
    
    # Ağırlıklı puan formülüne göre ağırlıklı puanı hesaplar ve döndürür.
    return (v / (v + m) * R) + (m / (m + v) * C)

# "q_movies" veri çerçevesine "weighted_rating" adında yeni bir sütun ekler.
# Bu sütun, "weight_rating" fonksiyonunu kullanarak her bir film için ağırlıklı puanı içerir.
q_movies["weighted_rating"] = q_movies.apply(weight_rating, axis=1)

# "q_movies" veri çerçevesini "score" sütununa göre büyükten küçüğe sıralar.
q_movies = q_movies.sort_values("score", ascending=False)


q_movies.head(10)
